# Demo So sánh Bảo mật: PostgreSQL vs. MongoDB

Notebook này so sánh trực tiếp các khái niệm bảo mật trên PostgreSQL và MongoDB.

## Các mô hình được Demo

Notebook này sẽ trình bày hai mô hình kiểm soát truy cập chính:

1.  **RBAC (Role-Based Access Control):** Kiểm soát truy cập dựa trên vai trò. Quyền được gán cho các "vai trò" (ví dụ: `admin`, `readonly_user`), và người dùng được gán vào các vai trò đó. Chúng ta sẽ thấy điều này trên cả PostgreSQL và MongoDB.

2.  **ABAC (Attribute-Based Access Control):** Kiểm soát truy cập dựa trên thuộc tính. Quyền truy cập được quyết định dựa trên các "thuộc tính" của người dùng, tài nguyên, hoặc môi trường. Chúng ta sẽ thấy một ví dụ điển hình của ABAC qua tính năng Row-Level Security của PostgreSQL.

### 1. Thiết lập & Kết nối Quản trị

Nhập thư viện và thiết lập kết nối ban đầu với cả hai DB bằng quyền quản trị.
**Lưu ý:** Sau khi chạy `docker-compose up -d`, hãy đợi khoảng 10-15 giây để các DB khởi động hoàn toàn trước khi chạy ô này.

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from pymongo import MongoClient
from pymongo.errors import OperationFailure

# --- Thông tin kết nối PostgreSQL ---
PG_ADMIN_USER = 'db_user'
PG_ADMIN_PASSWORD = 'db_password'
PG_HOST = 'localhost'
PG_PORT = '5433'
PG_DB = 'mycompany'
ADMIN_PG_CONN_STR = f'postgresql://{PG_ADMIN_USER}:{PG_ADMIN_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DB}'
admin_pg_engine = create_engine(ADMIN_PG_CONN_STR)
print("Kết nối PostgreSQL với tư cách quản trị viên thành công!")

# --- Thông tin kết nối MongoDB ---
MONGO_ADMIN_USER = 'mongo_user'
MONGO_ADMIN_PASSWORD = 'mongo_password'
MONGO_HOST = 'localhost'
MONGO_PORT = 27018
MONGO_DB = 'mycompany'
admin_mongo_client = MongoClient(host=MONGO_HOST, port=MONGO_PORT, username=MONGO_ADMIN_USER, password=MONGO_ADMIN_PASSWORD)
admin_mongo_db = admin_mongo_client[MONGO_DB]
print("Kết nối MongoDB với tư cách quản trị viên thành công!")

Kết nối PostgreSQL với tư cách quản trị viên thành công!
Kết nối MongoDB với tư cách quản trị viên thành công!


### 2. Tạo Dữ liệu

Tạo bảng/collection `employees` trong cả hai DB với dữ liệu mẫu giống nhau.

In [2]:
# --- PostgreSQL: Tạo bảng và chèn dữ liệu ---
with admin_pg_engine.connect() as connection:
    connection.execute(text('DROP TABLE IF EXISTS employees CASCADE;'))
    create_table_query = """
    CREATE TABLE employees (id SERIAL PRIMARY KEY, name VARCHAR(100), salary INT, department VARCHAR(50));
    INSERT INTO employees (name, salary, department) VALUES
    ('Alice', 70000, 'Engineering'), ('Bob', 80000, 'Engineering'), ('Charlie', 60000, 'HR'),
    ('David', 95000, 'Sales'), ('Eve', 105000, 'Sales');
    """
    connection.execute(text(create_table_query))
    connection.commit()
print("PostgreSQL: Bảng 'employees' đã được tạo.")
display(pd.read_sql('SELECT * FROM employees', admin_pg_engine))

# --- MongoDB: Tạo collection và chèn dữ liệu ---
admin_mongo_db.employees.drop()
employees_data = [
    {'id': 1, 'name': 'Alice', 'Salary': 70000, 'department': 'Engineering'},
    {'id': 2, 'name': 'Bob', 'Salary': 80000, 'department': 'Engineering'},
    {'id': 3, 'name': 'Charlie', 'Salary': 60000, 'department': 'HR'},
    {'id': 4, 'name': 'David', 'Salary': 95000, 'department': 'Sales'},
    {'id': 5, 'name': 'Eve', 'Salary': 105000, 'department': 'Sales'}
]
admin_mongo_db.employees.insert_many(employees_data)
print("\nMongoDB: Collection 'employees' đã được tạo.")
for doc in admin_mongo_db.employees.find():
    print(doc)

PostgreSQL: Bảng 'employees' đã được tạo.


,id,name,salary,department
0,1,Alice,70000,Engineering
1,2,Bob,80000,Engineering
2,3,Charlie,60000,HR
3,4,David,95000,Sales
4,5,Eve,105000,Sales



MongoDB: Collection 'employees' đã được tạo.
{'_id': ObjectId('691d92cbebd3321b7a28e9fd'), 'id': 1, 'name': 'Alice', 'Salary': 70000, 'department': 'Engineering'}
{'_id': ObjectId('691d92cbebd3321b7a28e9fe'), 'id': 2, 'name': 'Bob', 'Salary': 80000, 'department': 'Engineering'}
{'_id': ObjectId('691d92cbebd3321b7a28e9ff'), 'id': 3, 'name': 'Charlie', 'Salary': 60000, 'department': 'HR'}
{'_id': ObjectId('691d92cbebd3321b7a28ea00'), 'id': 4, 'name': 'David', 'Salary': 95000, 'department': 'Sales'}
{'_id': ObjectId('691d92cbebd3321b7a28ea01'), 'id': 5, 'name': 'Eve', 'Salary': 105000, 'department': 'Sales'}


### 3. RBAC - Người dùng chỉ đọc

**Mô hình:** RBAC (Role-Based Access Control).

**Mục tiêu:** Tạo một "vai trò" người dùng chỉ có quyền đọc.

In [3]:
# --- PostgreSQL: Tạo người dùng chỉ đọc ---
with admin_pg_engine.connect() as connection:
    connection.execute(text("DROP ROLE IF EXISTS readonly_user;"))
    connection.execute(text("CREATE USER readonly_user WITH PASSWORD 'readonly_pass';"))
    connection.execute(text('GRANT SELECT ON employees TO readonly_user;'))
    connection.commit()
print("PostgreSQL: Người dùng 'readonly_user' đã được tạo.")

# --- MongoDB: Tạo người dùng chỉ đọc ---
try:
    admin_mongo_db.command("dropUser", "mongo_readonly_user")
except OperationFailure:
    pass # Bỏ qua nếu người dùng không tồn tại
admin_mongo_db.command("createUser", "mongo_readonly_user", pwd="readonly_pass", roles=[{"role": "read", "db": MONGO_DB}])
print("MongoDB: Người dùng 'mongo_readonly_user' đã được tạo.")

PostgreSQL: Người dùng 'readonly_user' đã được tạo.
MongoDB: Người dùng 'mongo_readonly_user' đã được tạo.


#### Thử nghiệm RBAC

In [4]:
# --- PostgreSQL: Thử nghiệm ---
PG_READONLY_CONN_STR = f'postgresql://readonly_user:readonly_pass@{PG_HOST}:{PG_PORT}/{PG_DB}'
pg_readonly_engine = create_engine(PG_READONLY_CONN_STR)
print("--- PostgreSQL: Đọc với readonly_user (thành công) ---")
display(pd.read_sql('SELECT * FROM employees', pg_readonly_engine))
try:
    with pg_readonly_engine.connect() as connection:
        connection.execute(text('DELETE FROM employees WHERE id = 1;'))
        connection.commit()
except Exception as e:
    print("\n--- PostgreSQL: Xóa với readonly_user (thất bại như mong đợi) ---")
    print(e.orig)

# --- MongoDB: Thử nghiệm ---
mongo_readonly_client = MongoClient(host=MONGO_HOST, port=MONGO_PORT, username='mongo_readonly_user', password='readonly_pass', authSource=MONGO_DB)
readonly_db = mongo_readonly_client[MONGO_DB]
print("\n--- MongoDB: Đọc với mongo_readonly_user (thành công) ---")
for doc in readonly_db.employees.find():
    print(doc)
try:
    readonly_db.employees.insert_one({'name': 'Fail Test'})
except Exception as e:
    print("\n--- MongoDB: Chèn với mongo_readonly_user (thất bại như mong đợi) ---")
    print(e)

--- PostgreSQL: Đọc với readonly_user (thành công) ---


,id,name,salary,department
0,1,Alice,70000,Engineering
1,2,Bob,80000,Engineering
2,3,Charlie,60000,HR
3,4,David,95000,Sales
4,5,Eve,105000,Sales



--- PostgreSQL: Xóa với readonly_user (thất bại như mong đợi) ---
permission denied for table employees


--- MongoDB: Đọc với mongo_readonly_user (thành công) ---
{'_id': ObjectId('691d92cbebd3321b7a28e9fd'), 'id': 1, 'name': 'Alice', 'Salary': 70000, 'department': 'Engineering'}
{'_id': ObjectId('691d92cbebd3321b7a28e9fe'), 'id': 2, 'name': 'Bob', 'Salary': 80000, 'department': 'Engineering'}
{'_id': ObjectId('691d92cbebd3321b7a28e9ff'), 'id': 3, 'name': 'Charlie', 'Salary': 60000, 'department': 'HR'}
{'_id': ObjectId('691d92cbebd3321b7a28ea00'), 'id': 4, 'name': 'David', 'Salary': 95000, 'department': 'Sales'}
{'_id': ObjectId('691d92cbebd3321b7a28ea01'), 'id': 5, 'name': 'Eve', 'Salary': 105000, 'department': 'Sales'}

--- MongoDB: Chèn với mongo_readonly_user (thất bại như mong đợi) ---
not authorized on mycompany to execute command { insert: "employees", ordered: true, lsid: { id: UUID("9cab0389-f97d-45a6-b085-518ad68ed836") }, $db: "mycompany" }, full error: {'ok': 0.0, 'errmsg

### 4. Bảo mật cấp cột - Sử dụng View

**Mục tiêu:** Tạo một `view` trong mỗi DB để ẩn cột/trường `salary`.

In [5]:
# --- PostgreSQL: Tạo View và người dùng cho View ---
with admin_pg_engine.connect() as connection:
    connection.execute(text('CREATE OR REPLACE VIEW public_employees AS SELECT id, name, department FROM employees;'))
    connection.execute(text("DROP ROLE IF EXISTS view_user;"))
    connection.execute(text("CREATE USER view_user WITH PASSWORD 'view_pass';"))
    connection.execute(text('GRANT SELECT ON public_employees TO view_user;'))
    connection.commit()
print("PostgreSQL: View 'public_employees' và người dùng 'view_user' đã được tạo.")

# --- MongoDB: Tạo View và người dùng cho View ---
admin_mongo_db.drop_collection("public_employees_mongo")

admin_mongo_db.command("create", "public_employees_mongo", viewOn="employees", pipeline=[{"$project": {"Salary": 0}}])
try:
    admin_mongo_db.command("dropUser", "mongo_view_user")
except OperationFailure:
    pass
admin_mongo_db.command("createUser", "mongo_view_user", pwd="view_pass", roles=[{"role": "read", "db": MONGO_DB}])
print("MongoDB: View 'public_employees_mongo' và người dùng 'mongo_view_user' đã được tạo.")



PostgreSQL: View 'public_employees' và người dùng 'view_user' đã được tạo.
MongoDB: View 'public_employees_mongo' và người dùng 'mongo_view_user' đã được tạo.


#### Thử nghiệm người dùng View

In [6]:
# --- PostgreSQL: Thử nghiệm ---
PG_VIEW_CONN_STR = f'postgresql://view_user:view_pass@{PG_HOST}:{PG_PORT}/{PG_DB}'
pg_view_engine = create_engine(PG_VIEW_CONN_STR)
print("--- PostgreSQL: Đọc từ View với view_user (thành công) ---")
display(pd.read_sql('SELECT * FROM public_employees', pg_view_engine))
try:
    pd.read_sql('SELECT * FROM employees', pg_view_engine)
except Exception as e:
    print("\n--- PostgreSQL: Đọc từ bảng gốc với view_user (thất bại như mong đợi) ---")
    print(e.orig)

# --- MongoDB: Thử nghiệm ---
mongo_view_client = MongoClient(host=MONGO_HOST, port=MONGO_PORT, username='mongo_view_user', password='view_pass', authSource=MONGO_DB)
view_db = mongo_view_client[MONGO_DB]
print("\n--- MongoDB: Đọc từ View 'public_employees_mongo' với mongo_view_user ---")
for doc in view_db.public_employees_mongo.find():
    print(doc)

--- PostgreSQL: Đọc từ View với view_user (thành công) ---


,id,name,department
0,1,Alice,Engineering
1,2,Bob,Engineering
2,3,Charlie,HR
3,4,David,Sales
4,5,Eve,Sales



--- PostgreSQL: Đọc từ bảng gốc với view_user (thất bại như mong đợi) ---
permission denied for table employees


--- MongoDB: Đọc từ View 'public_employees_mongo' với mongo_view_user ---
{'_id': ObjectId('691d92cbebd3321b7a28e9fd'), 'id': 1, 'name': 'Alice', 'department': 'Engineering'}
{'_id': ObjectId('691d92cbebd3321b7a28e9fe'), 'id': 2, 'name': 'Bob', 'department': 'Engineering'}
{'_id': ObjectId('691d92cbebd3321b7a28e9ff'), 'id': 3, 'name': 'Charlie', 'department': 'HR'}
{'_id': ObjectId('691d92cbebd3321b7a28ea00'), 'id': 4, 'name': 'David', 'department': 'Sales'}
{'_id': ObjectId('691d92cbebd3321b7a28ea01'), 'id': 5, 'name': 'Eve', 'department': 'Sales'}


### 5. ABAC - Bảo mật cấp dòng (Row-Level Security)

**Mô hình:** ABAC (Attribute-Based Access Control).

**Mục tiêu:** Chỉ cho phép người dùng xem các nhân viên trong cùng phòng ban của họ. Quyền truy cập phụ thuộc vào **thuộc tính** của dữ liệu (`department`) và **thuộc tính** của người dùng (`current_user`).

RLS là một tính năng được tích hợp sẵn trong PostgreSQL. Trong MongoDB, để đạt được điều này cần các kỹ thuật phức tạp hơn và không phải là tính năng gốc. Do đó, phần này chỉ demo trên **PostgreSQL**.

In [7]:
with admin_pg_engine.connect() as connection:
         # Kích hoạt RLS trên bảng
        connection.execute(text('ALTER TABLE employees ENABLE ROW LEVEL SECURITY;'))
        # Xóa policy cũ nếu tồn tại
        connection.execute(text('DROP POLICY IF EXISTS department_policy ON employees;'))
         # Tạo policy, so sánh chữ thường để đảm bảo chính xác
        policy_query = "CREATE POLICY department_policy ON employees FOR SELECT USING (LOWER(department) = LOWER(current_user));"
        connection.execute(text(policy_query))
         
        # Tạo người dùng có tên khớp với một phòng ban (dùng chữ thường)
        connection.execute(text("DROP ROLE IF EXISTS engineering;"))
        connection.execute(text("CREATE USER engineering WITH PASSWORD 'eng_pass';"))
        connection.execute(text('GRANT SELECT ON employees TO engineering;'))
        connection.commit()
    
print("PostgreSQL: RLS đã được kích hoạt và policy đã được tạo.")

# --- MongoDB: Tạo View với $match theo department ---
admin_mongo_db.drop_collection("engineering_employees")

# Tạo view chỉ hiển thị nhân viên thuộc phòng ban 'engineering'
admin_mongo_db.command(
    "create",
    "engineering_employees",
    viewOn="employees",
    pipeline=[ {"$match": {"department": {"$regex": "^engineering$", "$options": "i"}}}
    ]
)

# Xóa user cũ nếu có
try:
    admin_mongo_db.command("dropUser", "engineering")
except OperationFailure:
    pass

# Tạo user 'engineering' chỉ có quyền đọc view này
admin_mongo_db.command(
    "createUser",
    "engineering",
    pwd="eng_pass",
    roles=[{"role": "read", "db": MONGO_DB}]
)

print("MongoDB: View 'engineering_employees' với $match và user 'engineering' đã được tạo.")


PostgreSQL: RLS đã được kích hoạt và policy đã được tạo.
MongoDB: View 'engineering_employees' với $match và user 'engineering' đã được tạo.


#### Thử nghiệm ABAC (RLS)

In [8]:
PG_RLS_CONN_STR = f'postgresql://engineering:eng_pass@{PG_HOST}:{PG_PORT}/{PG_DB}'
pg_rls_engine = create_engine(PG_RLS_CONN_STR)

print("--- PostgreSQL: Đọc dữ liệu với người dùng 'Engineering' (chỉ thấy phòng ban 'Engineering') ---")
display(pd.read_sql('SELECT * FROM employees', pg_rls_engine))

# --- MongoDB: Thử nghiệm với view $match ---

# Kết nối với user 'engineering'
engineering_client = MongoClient(
    host=MONGO_HOST,
    port=MONGO_PORT,
    username='engineering',
    password='eng_pass',
    authSource=MONGO_DB
)
eng_db = engineering_client[MONGO_DB]

print("\n--- MongoDB: Đọc từ View 'engineering_employees' với user 'engineering' ---")
for doc in eng_db.engineering_employees.find():
    print(doc)


--- PostgreSQL: Đọc dữ liệu với người dùng 'Engineering' (chỉ thấy phòng ban 'Engineering') ---


,id,name,salary,department
0,1,Alice,70000,Engineering
1,2,Bob,80000,Engineering



--- MongoDB: Đọc từ View 'engineering_employees' với user 'engineering' ---
{'_id': ObjectId('691d92cbebd3321b7a28e9fd'), 'id': 1, 'name': 'Alice', 'Salary': 70000, 'department': 'Engineering'}
{'_id': ObjectId('691d92cbebd3321b7a28e9fe'), 'id': 2, 'name': 'Bob', 'Salary': 80000, 'department': 'Engineering'}
